Análisis exploratorio 'Agricultura y Desarrollo Local EDA en Antiquia'

In [1]:
import funciones_mapa as func
import geopandas as gpd
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import mapclassify
import folium

In [2]:
# Lectura de los datos desde diferentes origenes
df_hasta_2000 = func.leer_archivo("datos_antes_2000.csv")
display(len(df_hasta_2000))

df_desde_2000 = func.leer_TablaBD("produccion_agricola_antioquia")
display(len(df_desde_2000))

df_hasta_2000["Área Total"] = df_hasta_2000["Área Total"].str.replace(",", ".").astype(float)
df_hasta_2000["Área Producción"] = df_hasta_2000["Área Producción"].str.replace(",", ".").astype(float)
df_hasta_2000["Volumen Producción"] = df_hasta_2000["Volumen Producción"].str.replace(",", ".").astype(float)

5564

c:\Users\juanr\Documents\especializacionEDA\agricultura-y-desarrollo-rural-EDA\ConexionBD.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT  * FROM dbo.{nombre_tabla}", conn)


28881

In [3]:
# Union de ambas fuentes de datos
df_completo = func.unir_data(df_hasta_2000, df_desde_2000)
display(len(df_completo))


34445

In [4]:
df_completo['Municipio'].unique()

<StringArray>
[           'Abejorral',             'Apartadó',           'Montebello',
        'Santa Bárbara',                'Turbo',              'Giraldo',
              'Cáceres',             'Caucasia',               'Mutatá',
                'Nechí',
 ...
           'La Pintada',             'Medellín',                'Bello',
           'Copacabana',            'Girardota',             'Sabaneta',
             'Envigado',               'Itagui',          'La Estrella',
 'El Carmen De Viboral']
Length: 126, dtype: str

In [5]:
#Tamaño del dataframe completo
df_completo.shape

(34445, 8)

In [6]:
#Tipos de datos
df_completo.info()

<class 'pandas.DataFrame'>
RangeIndex: 34445 entries, 0 to 34444
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Tipo                34445 non-null  str   
 1   Rubro               34445 non-null  str   
 2   Subregion           34445 non-null  str   
 3   Año                 34445 non-null  int64 
 4   Municipio           34445 non-null  str   
 5   Área Total          34403 non-null  object
 6   Área Producción     34395 non-null  object
 7   Volumen Producción  34395 non-null  object
dtypes: int64(1), object(3), str(4)
memory usage: 2.1+ MB


In [7]:
missing_count = df_completo.isnull().sum()
missing_count

Tipo                   0
Rubro                  0
Subregion              0
Año                    0
Municipio              0
Área Total            42
Área Producción       50
Volumen Producción    50
dtype: int64

In [8]:
df_completo_TipoDatos = func.conversion_Datos(df_completo, 'Área Total', 'float64')
df_completo_TipoDatos = func.conversion_Datos(df_completo, 'Área Producción', 'float64')
df_completo_TipoDatos = func.conversion_Datos(df_completo, 'Volumen Producción', 'float64')
df_completo_TipoDatos.info()

<class 'pandas.DataFrame'>
RangeIndex: 34445 entries, 0 to 34444
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Tipo                34445 non-null  str    
 1   Rubro               34445 non-null  str    
 2   Subregion           34445 non-null  str    
 3   Año                 34445 non-null  int64  
 4   Municipio           34445 non-null  str    
 5   Área Total          34403 non-null  float64
 6   Área Producción     34395 non-null  float64
 7   Volumen Producción  34395 non-null  float64
dtypes: float64(3), int64(1), str(4)
memory usage: 2.1 MB


In [9]:
df_agricola_estandar = func.estandarizar_nombres(df_completo, 'Municipio')

Estandarizando textos en la columna: 'Municipio'...
¡Estandarización completada!


In [10]:
ruta_mapa = 'datos_espaciales\MGN2025_MPIO_GRAFICO\MGN_ADM_MPIO_GRAFICO.shp'
mapa_colombia= gpd.read_file(ruta_mapa)
display(mapa_colombia)

<>:1: SyntaxWarning: "\M" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\M"? A raw string is also an option.
<>:1: SyntaxWarning: "\M" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\M"? A raw string is also an option.
C:\Users\juanr\AppData\Local\Temp\ipykernel_8360\4081563783.py:1: SyntaxWarning: "\M" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\M"? A raw string is also an option.
  ruta_mapa = 'datos_espaciales\MGN2025_MPIO_GRAFICO\MGN_ADM_MPIO_GRAFICO.shp'


,dpto_ccdgo,mpio_ccdgo,mpio_cdpmp,dpto_cnmbr,mpio_cnmbr,mpio_crslc,mpio_tipo,mpio_narea,mpio_nano,shape_Leng,shape_Area,geometry
0,05,001,05001,ANTIOQUIA,MEDELLÍN,1965,MUNICIPIO,374.741484,2025,1.035380,0.030608,"POLYGON ((-75.4857 6.20163, -75.48567 6.20148,..."
1,05,002,05002,ANTIOQUIA,ABEJORRAL,1814,MUNICIPIO,506.959782,2024,1.158504,0.041384,"POLYGON ((-75.46938 5.94575, -75.46897 5.94571..."
2,05,004,05004,ANTIOQUIA,ABRIAQUÍ,1912,MUNICIPIO,296.974188,2025,0.810667,0.024253,"POLYGON ((-76.09047 6.74542, -76.09066 6.74548..."
3,05,021,05021,ANTIOQUIA,ALEJANDRÍA,Decreto departamental 304 de 1907,MUNICIPIO,128.856431,2024,0.705200,0.010535,"POLYGON ((-75.0332 6.41586, -75.03313 6.41585,..."
4,05,030,05030,ANTIOQUIA,AMAGÁ,1912,MUNICIPIO,84.117145,2024,0.445533,0.006867,"POLYGON ((-75.67587 6.08561, -75.6754 6.08491,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1117,97,889,97889,VAUPÉS,YAVARATÉ,Decreto Nacional 1510 del 29 de Diciembre de 1914,ÁREA NO MUNICIPALIZADA,4640.894609,2024,3.970046,0.376344,"POLYGON ((-69.7021 1.10907, -69.70233 1.10833,..."
1118,99,001,99001,VICHADA,PUERTO CARREÑO,Decreto 1594 de Ago 5 de 1974,MUNICIPIO,12142.323352,2024,5.475145,0.985914,"POLYGON ((-67.80972 6.32432, -67.80946 6.32432..."
1119,99,524,99524,VICHADA,LA PRIMAVERA,Decreto 676 de Abril13 de 1987,MUNICIPIO,18492.107241,2024,8.081556,1.506227,"POLYGON ((-68.81353 6.17289, -68.81359 6.15848..."
1120,99,624,99624,VICHADA,SANTA ROSALÍA,Ordenanza 19 de Noviembre 26 de 1993,MUNICIPIO,3679.005021,2024,3.805847,0.299960,"POLYGON ((-70.65378 5.37298, -70.65327 5.37298..."


In [11]:
df_municipios_anti=mapa_colombia[mapa_colombia['dpto_ccdgo'] =='05']
display(df_municipios_anti)

,dpto_ccdgo,mpio_ccdgo,mpio_cdpmp,dpto_cnmbr,mpio_cnmbr,mpio_crslc,mpio_tipo,mpio_narea,mpio_nano,shape_Leng,shape_Area,geometry
0,05,001,05001,ANTIOQUIA,MEDELLÍN,1965,MUNICIPIO,374.741484,2025,1.035380,0.030608,"POLYGON ((-75.4857 6.20163, -75.48567 6.20148,..."
1,05,002,05002,ANTIOQUIA,ABEJORRAL,1814,MUNICIPIO,506.959782,2024,1.158504,0.041384,"POLYGON ((-75.46938 5.94575, -75.46897 5.94571..."
2,05,004,05004,ANTIOQUIA,ABRIAQUÍ,1912,MUNICIPIO,296.974188,2025,0.810667,0.024253,"POLYGON ((-76.09047 6.74542, -76.09066 6.74548..."
3,05,021,05021,ANTIOQUIA,ALEJANDRÍA,Decreto departamental 304 de 1907,MUNICIPIO,128.856431,2024,0.705200,0.010535,"POLYGON ((-75.0332 6.41586, -75.03313 6.41585,..."
4,05,030,05030,ANTIOQUIA,AMAGÁ,1912,MUNICIPIO,84.117145,2024,0.445533,0.006867,"POLYGON ((-75.67587 6.08561, -75.6754 6.08491,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
120,05,885,05885,ANTIOQUIA,YALÍ,Ordenanza 11 de Noviembrebre 12 de 1959,MUNICIPIO,440.552085,2025,1.420701,0.036056,"POLYGON ((-74.57896 6.78189, -74.57901 6.7816,..."
121,05,887,05887,ANTIOQUIA,YARUMAL,1911,MUNICIPIO,712.322657,2024,1.736343,0.058278,"POLYGON ((-75.31703 7.23923, -75.31698 7.23923..."
122,05,890,05890,ANTIOQUIA,YOLOMBÓ,Decreto departamental 384 del 12 de Abril de 1883,MUNICIPIO,943.344322,2025,2.923176,0.077179,"POLYGON ((-74.92369 6.74878, -74.92359 6.74867..."
123,05,893,05893,ANTIOQUIA,YONDÓ,Ordenanza 38 de Noviembrebre 23 de 1978,MUNICIPIO,1892.713928,2025,2.348293,0.155050,"POLYGON ((-73.9235 7.21346, -73.92426 7.21034,..."


In [12]:
df_municipios_estandar = func.estandarizar_nombres(df_municipios_anti,'mpio_cnmbr')

Estandarizando textos en la columna: 'mpio_cnmbr'...
¡Estandarización completada!


In [13]:
reemplazos = {
    "DON MATIAS": "DONMATIAS",
    "EL PENOL": "PENOL",
    "EL RETIRO": "RETIRO",
    "SAN ANDRES DE C.": "SAN ANDRES DE CUERQUIA",
    "SAN PEDRO DE LOS M.": "SAN PEDRO DE LOS MILAGROS",
    "SAN VICENTE": "SAN VICENTE FERRER"
}

df_agricola_estandar =func.reemplazar_nombre_municipios(df_agricola_estandar, "Municipio", reemplazos)


In [14]:
df_municipios_coordenadas=pd.merge(df_agricola_estandar,df_municipios_estandar,how="inner",
    left_on="Municipio",
    right_on="mpio_cnmbr" )

display(df_municipios_coordenadas)

,Tipo,Rubro,Subregion,Año,Municipio,Área Total,Área Producción,Volumen Producción,dpto_ccdgo,mpio_ccdgo,mpio_cdpmp,dpto_cnmbr,mpio_cnmbr,mpio_crslc,mpio_tipo,mpio_narea,mpio_nano,shape_Leng,shape_Area,geometry
0,Permanentes,Aguacate,Oriente,1990,ABEJORRAL,45.0,23.0,138.0,05,002,05002,ANTIOQUIA,ABEJORRAL,1814,MUNICIPIO,506.959782,2024,1.158504,0.041384,"POLYGON ((-75.46938 5.94575, -75.46897 5.94571..."
1,Permanentes,Aguacate,Urabá,1990,APARTADO,29.0,27.0,405.0,05,045,05045,ANTIOQUIA,APARTADO,Ordenanza 7 de Noviembre 30 de 1967,MUNICIPIO,535.351357,2024,1.285608,0.043800,"POLYGON ((-76.41162 8.00712, -76.41157 8.00695..."
2,Permanentes,Aguacate,Suroeste,1990,MONTEBELLO,17.0,12.0,78.0,05,467,05467,ANTIOQUIA,MONTEBELLO,Ordenanza 44 de Abril 28 de 1913,MUNICIPIO,75.998296,2024,0.472159,0.006204,"POLYGON ((-75.57878 5.97215, -75.57851 5.97215..."
3,Permanentes,Aguacate,Suroeste,1990,SANTA BARBARA,150.0,140.0,980.0,05,679,05679,ANTIOQUIA,SANTA BARBARA,1822,MUNICIPIO,196.118138,2024,0.852976,0.016008,"POLYGON ((-75.59353 6.00584, -75.59309 6.0058,..."
4,Permanentes,Aguacate,Urabá,1990,TURBO,75.0,55.0,825.0,05,837,05837,ANTIOQUIA,TURBO,1840,MUNICIPIO,2923.242082,2025,6.639314,0.239141,"POLYGON ((-76.40265 8.46347, -76.40281 8.46367..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34440,Transitorios,Zanahoria,Oriente,2023,LA UNION,458.0,458.0,458.0,05,400,05400,ANTIOQUIA,LA UNION,Ordenanza 18 de Abril de 1911,MUNICIPIO,167.670338,2024,0.679671,0.013692,"POLYGON ((-75.34644 6.03374, -75.34628 6.03369..."
34441,Transitorios,Zanahoria,Oriente,2023,MARINILLA,190.0,190.0,190.0,05,440,05440,ANTIOQUIA,MARINILLA,1787,MUNICIPIO,112.236046,2025,0.599432,0.009170,"POLYGON ((-75.35173 6.15142, -75.35174 6.15143..."
34442,Transitorios,Zanahoria,Valle de Aburra,2023,MEDELLIN,3.0,3.0,3.0,05,001,05001,ANTIOQUIA,MEDELLIN,1965,MUNICIPIO,374.741484,2025,1.035380,0.030608,"POLYGON ((-75.4857 6.20163, -75.48567 6.20148,..."
34443,Transitorios,Zanahoria,Norte,2023,SAN PEDRO DE LOS MILAGROS,202.0,202.0,202.0,05,664,05664,ANTIOQUIA,SAN PEDRO DE LOS MILAGROS,1813,MUNICIPIO,235.249084,2025,0.870273,0.019223,"POLYGON ((-75.56355 6.52905, -75.56351 6.52902..."


In [15]:
df_mapa_agrupado = df_municipios_coordenadas.groupby('Municipio').agg({
    'Volumen Producción': 'sum', 
    'geometry': 'first'         # ¡Esto salva nuestro mapa!
}).reset_index()
df_mapa_agrupado = gpd.GeoDataFrame(df_mapa_agrupado, geometry='geometry')

In [16]:
display(df_mapa_agrupado)

,Municipio,Volumen Producción,geometry
0,ABEJORRAL,506677.86,"POLYGON ((-75.46938 5.94575, -75.46897 5.94571..."
1,ABRIAQUI,14965.12,"POLYGON ((-76.09047 6.74542, -76.09066 6.74548..."
2,ALEJANDRIA,39561.39,"POLYGON ((-75.0332 6.41586, -75.03313 6.41585,..."
3,AMAGA,119287.44,"POLYGON ((-75.67587 6.08561, -75.6754 6.08491,..."
4,AMALFI,194752.18,"POLYGON ((-74.91364 7.28807, -74.91361 7.28789..."
...,...,...,...
120,YALI,431242.00,"POLYGON ((-74.57896 6.78189, -74.57901 6.7816,..."
121,YARUMAL,126821.76,"POLYGON ((-75.31703 7.23923, -75.31698 7.23923..."
122,YOLOMBO,318477.63,"POLYGON ((-74.92369 6.74878, -74.92359 6.74867..."
123,YONDO,440389.44,"POLYGON ((-73.9235 7.21346, -73.92426 7.21034,..."


In [ ]:
df_mapa_agrupado['geometry'] = df_mapa_agrupado['geometry'].simplify(0.005, preserve_topology=True)

df_mapa_agrupado['Producción (Detalle)'] = df_mapa_agrupado['Volumen Producción'].apply(
    lambda x: f"{x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
)


mapa_interactivo = df_mapa_agrupado.explore(
    column='Volumen Producción',       
    cmap='YlGn',                    
    scheme='quantiles',             
    k=3,                            
    
    tooltip=['Municipio', 'Volumen Producción'], 
    tooltip_kwds=dict(labels=True),           
    
    popup=True,                     
    tiles='CartoDB positron',      
    legend_kwds={'caption': 'Producción Total (Terciles)'} 
)

mapa_interactivo = df_mapa_agrupado.explore(
    column='Volumen Producción',       
    cmap='YlGn',                    
    scheme='quantiles',           
    k=3,                          
    

    tooltip=['Municipio', 'Volumen Producción'], 
    tooltip_kwds=dict(labels=True),           
    
    popup=True,                   
    tiles='CartoDB positron',      
    legend_kwds={'caption': 'Producción Total (Terciles)'} 
)

mapa_interactivo

Generando el mapa interactivo de Antioquia...
2. Aplicando formato a los números...
